In [42]:
import pandas as pd
from tqdm import tqdm

In [9]:
threshold = '0.6'
dataset = 'structural_similarity'

test = pd.read_csv(f'./{dataset}/test_rows_structural.csv', index_col=0)
train = pd.read_csv(f'./{dataset}/train_rows_structural_{threshold}.csv', index_col=0)

In [10]:
train.loc[train.spectrum_id.isin(test.spectrum_id)]

,scan,spectrum_id,collision_energy,Adduct,Compound_Source,Compund_Name,Precursor_MZ,ExactMass,Charge,Ion_Mode,...,msManufacturer,msMassAnalyzer,msIonisation,msDissociationMethod,GNPS_library_membership,ppmBetweenExpAndThMass,Morgan_2048_2,Morgan_4096_2,Morgan_2048_3,Morgan_4096_3


In [11]:
test.loc[test.spectrum_id.isin(train.spectrum_id)]

,scan,spectrum_id,collision_energy,Adduct,Compound_Source,Compund_Name,Precursor_MZ,ExactMass,Charge,Ion_Mode,...,msManufacturer,msMassAnalyzer,msIonisation,msDissociationMethod,GNPS_library_membership,ppmBetweenExpAndThMass,Morgan_2048_2,Morgan_4096_2,Morgan_2048_3,Morgan_4096_3


In [12]:
train.columns

Index(['scan', 'spectrum_id', 'collision_energy', 'Adduct', 'Compound_Source',
       'Compund_Name', 'Precursor_MZ', 'ExactMass', 'Charge', 'Ion_Mode',
       'Smiles', 'INCHI', 'InChIKey_smiles', 'msManufacturer',
       'msMassAnalyzer', 'msIonisation', 'msDissociationMethod',
       'GNPS_library_membership', 'ppmBetweenExpAndThMass', 'Morgan_2048_2',
       'Morgan_4096_2', 'Morgan_2048_3', 'Morgan_4096_3'],
      dtype='object')

In [29]:
# Check for overlap using Morgan Fingerprints
from rdkit.Chem import AllChem
from rdkit.DataStructs.cDataStructs import ExplicitBitVect
from rdkit.DataStructs.cDataStructs import BulkTanimotoSimilarity, CreateFromBitString

from rdkit import DataStructs

train.Morgan_2048_2 = train.Morgan_2048_2.apply(lambda x:  CreateFromBitString(''.join(str(y) for y in x)))
train.Morgan_2048_3 = train.Morgan_2048_3.apply(lambda x:  CreateFromBitString(''.join(str(y) for y in x)))
train.Morgan_4096_2 = train.Morgan_4096_2.apply(lambda x:  CreateFromBitString(''.join(str(y) for y in x)))
train.Morgan_4096_3 = train.Morgan_4096_3.apply(lambda x:  CreateFromBitString(''.join(str(y) for y in x)))

test.Morgan_2048_2 = test.Morgan_2048_2.apply(lambda x:  CreateFromBitString(''.join(str(y) for y in x)))
test.Morgan_2048_3 = test.Morgan_2048_3.apply(lambda x:  CreateFromBitString(''.join(str(y) for y in x)))
test.Morgan_4096_2 = test.Morgan_4096_2.apply(lambda x:  CreateFromBitString(''.join(str(y) for y in x)))
test.Morgan_4096_3 = test.Morgan_4096_3.apply(lambda x:  CreateFromBitString(''.join(str(y) for y in x)))

In [43]:
sims = []
train_fps = train.Morgan_2048_2
test_fps = list(test.Morgan_2048_2)

for idx in tqdm(range(len(train))):
    sims.extend(DataStructs.BulkTanimotoSimilarity(train_fps.iloc[idx], test_fps))

100%|██████████| 122592/122592 [18:25<00:00, 110.90it/s]


In [44]:
pd.Series(sims).describe()

count    5.351018e+09
mean     1.054339e-01
std      6.783394e-02
min      0.000000e+00
25%      6.578947e-02
50%      9.278351e-02
75%      1.267606e-01
max      1.000000e+00
dtype: float64